## Reads in H1 DNN training and generates an output root TTree with DDN output added

This version saves all of the selected branches from the input TTree that go into the pandas df, all of the derived variables, and the DNN output.

In [1]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.colors import LogNorm
from matplotlib import rc
from numpy import inf
import os

from os import listdir


import uproot3

import matplotlib as mpl

from datetime import datetime
import subprocess

mpl.rcParams.update({'font.size': 19})
mpl.rcParams.update({'xtick.labelsize': 18}) 
mpl.rcParams.update({'ytick.labelsize': 18}) 
mpl.rcParams.update({'text.usetex' : False})
mpl.rcParams.update({'axes.labelsize': 18}) 
mpl.rcParams.update({'legend.frameon': False}) 



In [2]:
has_gpu = True


regression_training_name = 'training_h1_reg_v4b'

#data_dir = '/data/owen/DIS-reco/h1-fullsim-2021-09-27-v2a'
#data_dir = '/data/owen/DIS-reco/h1-fullsim-2021-10-12-v4a'
data_dir = '/data/owen/DIS-reco/h1-2021-10-14-v5f'

#dataset_type = 'Rapgap'
dataset_type = 'Data'

if dataset_type == 'Rapgap' :
    input_file = '%s/all-h1-rapgap.root' % data_dir

if dataset_type == 'Data' :
    input_file = '%s/all-h1-data.root' % data_dir




output_root_file = '%s/dnn-output-h1-v2-%s.root' % (data_dir, dataset_type)

print('\n\n Saving output in %s\n\n' % output_root_file )


#-- for testing
#max_events =   10000
#max_events =   100000

#-- for all events
max_events = 1e9









 Saving output in /data/owen/DIS-reco/h1-2021-10-14-v5f/dnn-output-h1-v2-Data.root




### Read in the input TTree.

The input ttree variables should be fairly self explanatory from the names.

The obs_x, obs_y, and obs_Q2 are arrays that contain the following calculations:

0 - electron

1 - E0 E Sigma

2 - E0 Theta Sigma

3 - DA

4 - hadron

5 - ISigma

6 - IDA

7 - Theta Sigma gamma

8 - eSigma

They are there for convenience to compare the DNN predictions with the standard methods.  They are not inputs to the DNN, so don't worry about them if you don't need or want them.


In [3]:
method_names = {}
method_names['[0]'] = 'e'
method_names['[1]'] = 'E0ESigma'
method_names['[2]'] = 'E0ThetaSigma'
method_names['[3]'] = 'DA'
method_names['[4]'] = 'h'
method_names['[5]'] = 'ISigma'
method_names['[6]'] = 'IDA'
method_names['[7]'] = 'ThetaSigmagamma'
method_names['[8]'] = 'eSigma'

for m in method_names :
    print( ' %s is %s' % (m, method_names[m]))

 [0] is e
 [1] is E0ESigma
 [2] is E0ThetaSigma
 [3] is DA
 [4] is h
 [5] is ISigma
 [6] is IDA
 [7] is ThetaSigmagamma
 [8] is eSigma


In [4]:
%%time


ur_file = uproot3.open(input_file)

print (ur_file.keys()) 
ur_tree = ur_file['%s/minitree' % dataset_type ]
print(ur_tree)
ur_tree.show()

selected_branches = []

if dataset_type == 'Rapgap' or dataset_type == 'Django' :
    selected_branches =     [
            'has_isr','has_fsr',
             'tower_sum_40','n_towers_40', ''
             'eta_pho_closest_to_ebeam','e_pho_closest_to_ebeam', 'phi_pho_closest_to_ebeam',
             'obs_x', 'obs_y', 'obs_Q2',
             'from_tlv_gen_Q2','from_tlv_gen_x','from_tlv_gen_y',
             'obs_e_e','obs_e_pz','obs_e_pt','obs_e_phi',
             'obs_hfs_e','obs_hfs_pz','obs_hfs_pt','obs_hfs_phi',
             'obs_dphi',
             'Empz', 'obs_e_trk_e',
             'beam_e_e', 'beam_p_e',
             'wgt'
            ]
else :
    selected_branches =     [
         'tower_sum_40','n_towers_40', ''
         'eta_pho_closest_to_ebeam','e_pho_closest_to_ebeam', 'phi_pho_closest_to_ebeam',
         'obs_x', 'obs_y', 'obs_Q2',
         'obs_e_e','obs_e_pz','obs_e_pt','obs_e_phi',
         'obs_hfs_e','obs_hfs_pz','obs_hfs_pt','obs_hfs_phi',
         'obs_dphi',
         'Empz', 'obs_e_trk_e',
         'beam_e_e', 'beam_p_e',
        ]
    
pandas_df   =  ur_tree.pandas.df( selected_branches, entrystop=max_events, flatten=True )

print('\n\n Number of entries in pandas_df:  %d ' % pandas_df.shape[0] )

[b'Data;1']
<TTree b'minitree' at 0x7fefd0277460>
wgt                        (no streamer)              asdtype('>f4')
Empz                       (no streamer)              asdtype('>f4')
from_tlv_gen_Q2            (no streamer)              asdtype('>f4')
from_tlv_gen_x             (no streamer)              asdtype('>f4')
from_tlv_gen_y             (no streamer)              asdtype('>f4')
beam_e_e                   (no streamer)              asdtype('>f4')
beam_p_e                   (no streamer)              asdtype('>f4')
has_isr                    (no streamer)              asdtype('int8')
has_fsr                    (no streamer)              asdtype('int8')
gen_e_e                    (no streamer)              asdtype('>f4')
gen_e_pz                   (no streamer)              asdtype('>f4')
gen_e_pt                   (no streamer)              asdtype('>f4')
gen_e_phi                  (no streamer)              asdtype('>f4')
gen_e_eta                  (no streamer)           

### Add any derived variables here

In [5]:
%%time

pandas_df.eval( 'obs_hfs_Empz = obs_hfs_e - obs_hfs_pz', inplace=True )
pandas_df.eval( 'obs_e_Empz = obs_e_e - obs_e_pz', inplace=True )

pandas_df.eval( 'obs_event_Empz = obs_hfs_Empz + obs_e_Empz', inplace=True )

pandas_df.eval( 'rot_pt1 = 0.70710678 * obs_hfs_pt - 0.70710678 * obs_e_pt', inplace=True )
pandas_df.eval( 'rot_pt2 = 0.70710678 * obs_hfs_pt + 0.70710678 * obs_e_pt', inplace=True )

pandas_df.eval( 'rot_Empz1 = 0.70710678 * obs_hfs_Empz - 0.70710678 * obs_e_Empz', inplace=True )
pandas_df.eval( 'rot_Empz2 = 0.70710678 * obs_hfs_Empz + 0.70710678 * obs_e_Empz', inplace=True )

pandas_df.eval( 'e_ecal_over_trk_ratio = tower_sum_40/obs_e_trk_e', inplace=True )
pandas_df.eval( 'e_ecal_over_trk_ratio = (e_ecal_over_trk_ratio<4)*e_ecal_over_trk_ratio + (e_ecal_over_trk_ratio>4)*4', inplace=True )

pandas_df.eval( 'dphi_pho_closest_to_ebeam = obs_e_phi - phi_pho_closest_to_ebeam', inplace=True )
pandas_df.eval( 'dphi_pho_closest_to_ebeam = (abs(dphi_pho_closest_to_ebeam)<3.14159265)*(dphi_pho_closest_to_ebeam)+(dphi_pho_closest_to_ebeam>3.14159265)*(dphi_pho_closest_to_ebeam-2*3.14159265) + (dphi_pho_closest_to_ebeam<-3.14159265)*(dphi_pho_closest_to_ebeam+2*3.14159265)', inplace=True )
pandas_df.eval( 'dphi_pho_closest_to_ebeam = (dphi_pho_closest_to_ebeam>0)*dphi_pho_closest_to_ebeam + (dphi_pho_closest_to_ebeam<0)*(dphi_pho_closest_to_ebeam+2*3.14159265)', inplace=True )
pandas_df.eval( 'dphi_pho_closest_to_ebeam = (phi_pho_closest_to_ebeam!=0)*(dphi_pho_closest_to_ebeam)+(phi_pho_closest_to_ebeam==0)*(-1)', inplace=True )

pandas_df.eval( 'e_pho_closest_to_ebeam = (e_pho_closest_to_ebeam<30)*e_pho_closest_to_ebeam + (e_pho_closest_to_ebeam>30)*30', inplace=True )

pandas_df.eval( 'n_towers_40 = (n_towers_40<7)*n_towers_40 + (n_towers_40>=7)*7', inplace=True  )


CPU times: user 62.7 ms, sys: 30.7 ms, total: 93.4 ms
Wall time: 65.7 ms


In [6]:
%%time

    

pandas_df.eval( 'obs_ptbal = 1. - obs_e_pt / obs_hfs_pt', inplace=True )
pandas_df.eval( 'obs_pzbal = 1. - (obs_hfs_Empz + obs_e_Empz)/2./beam_e_e', inplace=True )




pandas_df.eval( 'obs_hfs_theta = arctan2(obs_hfs_pt,obs_hfs_pz)', inplace=True )


CPU times: user 7.08 ms, sys: 14.5 ms, total: 21.6 ms
Wall time: 10.8 ms


## Apply any event selection here.

In [7]:
%%time

print('\n\n Number of entries in pandas_df before selection :  %d ' % pandas_df.shape[0] )


pandas_df = pandas_df.query('Empz > 0')


pandas_df = pandas_df.query('obs_event_Empz > 46 and obs_event_Empz < 62')

if dataset_type == 'Rapgap' or dataset_type == 'Django' :
    pandas_df = pandas_df.query('from_tlv_gen_Q2 > 200')

pandas_df = pandas_df.query('obs_hfs_pt > 0')

pandas_df = pandas_df.query('e_ecal_over_trk_ratio > 0')

print('\n\n Number of entries in pandas_df after selection:  %d ' % pandas_df.shape[0] )







 Number of entries in pandas_df before selection :  355805 


 Number of entries in pandas_df after selection:  334093 
CPU times: user 72.9 ms, sys: 36.3 ms, total: 109 ms
Wall time: 106 ms


## Set up machine learning stuff

In [8]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from pickle import load

#-- Ben suggested to try this.  2021-08-07
from tensorflow.keras.callbacks import EarlyStopping
earlystopping = EarlyStopping(patience=10,
               verbose=True,
               restore_best_weights=True)

import os

print(tf.config.list_physical_devices())

if has_gpu :
    os.environ['CUDA_VISIBLE_DEVICES']="0"
    physical_devices = tf.config.list_physical_devices('GPU') 
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

#####physical_devices = tf.config.list_physical_devices('CPU')

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Load the inputs for the DNN and transform them.   Don't change anything here!

In [9]:

X = np.c_[
    pandas_df['e_ecal_over_trk_ratio'].to_numpy(),
    pandas_df['n_towers_40'].to_numpy(),
    pandas_df['eta_pho_closest_to_ebeam'].to_numpy(),
    pandas_df['e_pho_closest_to_ebeam'].to_numpy(),
    pandas_df['dphi_pho_closest_to_ebeam'].to_numpy(),
    pandas_df['obs_e_pz'].to_numpy(),
    pandas_df['obs_e_e'].to_numpy(),
    pandas_df['obs_hfs_pz'].to_numpy(),
    pandas_df['obs_hfs_e'].to_numpy(),
    pandas_df['rot_pt1'].to_numpy(),
    pandas_df['rot_Empz1'].to_numpy(),
    pandas_df['rot_pt2'].to_numpy(),
    pandas_df['obs_pzbal'].to_numpy(),
    pandas_df['obs_ptbal'].to_numpy(),
    pandas_df['obs_dphi'].to_numpy(),
]


#-- Load the scaler transformations!  These are essential when reusing the training with a different dataset.

scaler = load( open('%s-scalers/input_scaler.pkl' % regression_training_name, 'rb'))
X = scaler.transform(X)

    

## Set up the regression network

In [10]:
model_r = tf.keras.models.load_model('%s_regression' % regression_training_name )
model_r.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                1024      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               66048     
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              5

In [11]:
%%time

mypreds_r = model_r.predict(X,batch_size=1000)

CPU times: user 613 ms, sys: 133 ms, total: 746 ms
Wall time: 670 ms


### Undo the variable transformations

In [12]:
scalerY = load( open('%s-scalers/target_scaler.pkl' % regression_training_name , 'rb'))

inv_trans_pred = scalerY.inverse_transform(mypreds_r)
pred_vals = np.exp( inv_trans_pred )

### Save the results in an output root file

This saves all of the variables in the pandas_df, including the derived ones, in addition to the DNN outputs.

In [13]:
pandas_df.keys()

Index(['tower_sum_40', 'n_towers_40', 'eta_pho_closest_to_ebeam',
       'e_pho_closest_to_ebeam', 'phi_pho_closest_to_ebeam', 'obs_x[0]',
       'obs_x[1]', 'obs_x[2]', 'obs_x[3]', 'obs_x[4]', 'obs_x[5]', 'obs_x[6]',
       'obs_x[7]', 'obs_x[8]', 'obs_y[0]', 'obs_y[1]', 'obs_y[2]', 'obs_y[3]',
       'obs_y[4]', 'obs_y[5]', 'obs_y[6]', 'obs_y[7]', 'obs_y[8]', 'obs_Q2[0]',
       'obs_Q2[1]', 'obs_Q2[2]', 'obs_Q2[3]', 'obs_Q2[4]', 'obs_Q2[5]',
       'obs_Q2[6]', 'obs_Q2[7]', 'obs_Q2[8]', 'obs_e_e', 'obs_e_pz',
       'obs_e_pt', 'obs_e_phi', 'obs_hfs_e', 'obs_hfs_pz', 'obs_hfs_pt',
       'obs_hfs_phi', 'obs_dphi', 'Empz', 'obs_e_trk_e', 'beam_e_e',
       'beam_p_e', 'obs_hfs_Empz', 'obs_e_Empz', 'obs_event_Empz', 'rot_pt1',
       'rot_pt2', 'rot_Empz1', 'rot_Empz2', 'e_ecal_over_trk_ratio',
       'dphi_pho_closest_to_ebeam', 'obs_ptbal', 'obs_pzbal', 'obs_hfs_theta'],
      dtype='object')

### Add the DNN outputs to the pandas data frame

In [14]:
pandas_df['dnn_x'] = pred_vals[:,0]
pandas_df['dnn_y'] = pred_vals[:,2]
pandas_df['dnn_Q2'] = pred_vals[:,1]

In [15]:
pandas_df.keys()

Index(['tower_sum_40', 'n_towers_40', 'eta_pho_closest_to_ebeam',
       'e_pho_closest_to_ebeam', 'phi_pho_closest_to_ebeam', 'obs_x[0]',
       'obs_x[1]', 'obs_x[2]', 'obs_x[3]', 'obs_x[4]', 'obs_x[5]', 'obs_x[6]',
       'obs_x[7]', 'obs_x[8]', 'obs_y[0]', 'obs_y[1]', 'obs_y[2]', 'obs_y[3]',
       'obs_y[4]', 'obs_y[5]', 'obs_y[6]', 'obs_y[7]', 'obs_y[8]', 'obs_Q2[0]',
       'obs_Q2[1]', 'obs_Q2[2]', 'obs_Q2[3]', 'obs_Q2[4]', 'obs_Q2[5]',
       'obs_Q2[6]', 'obs_Q2[7]', 'obs_Q2[8]', 'obs_e_e', 'obs_e_pz',
       'obs_e_pt', 'obs_e_phi', 'obs_hfs_e', 'obs_hfs_pz', 'obs_hfs_pt',
       'obs_hfs_phi', 'obs_dphi', 'Empz', 'obs_e_trk_e', 'beam_e_e',
       'beam_p_e', 'obs_hfs_Empz', 'obs_e_Empz', 'obs_event_Empz', 'rot_pt1',
       'rot_pt2', 'rot_Empz1', 'rot_Empz2', 'e_ecal_over_trk_ratio',
       'dphi_pho_closest_to_ebeam', 'obs_ptbal', 'obs_pzbal', 'obs_hfs_theta',
       'dnn_x', 'dnn_y', 'dnn_Q2'],
      dtype='object')

In [16]:
branch_dict = {}
data_dict = {}

for k in pandas_df.keys() :
    dict_key = k
    for m in method_names :
        if m in k :
            print( 'found %s in %s' % (m, k))
            dict_key = k.replace( m, '_%s' % method_names[m])
    dict_key = dict_key.replace('[','').replace(']','')  # in case not in method_names
    print( ' key  %s , dict_key %s' % (k, dict_key) )
    print( ' dtype for %s is ' % k, pandas_df[k].dtype)
    branch_dict[dict_key] = pandas_df[k].dtype
    data_dict[dict_key] = pandas_df[k].to_numpy()
    
    

 key  tower_sum_40 , dict_key tower_sum_40
 dtype for tower_sum_40 is  float32
 key  n_towers_40 , dict_key n_towers_40
 dtype for n_towers_40 is  int64
 key  eta_pho_closest_to_ebeam , dict_key eta_pho_closest_to_ebeam
 dtype for eta_pho_closest_to_ebeam is  float32
 key  e_pho_closest_to_ebeam , dict_key e_pho_closest_to_ebeam
 dtype for e_pho_closest_to_ebeam is  float64
 key  phi_pho_closest_to_ebeam , dict_key phi_pho_closest_to_ebeam
 dtype for phi_pho_closest_to_ebeam is  float32
found [0] in obs_x[0]
 key  obs_x[0] , dict_key obs_x_e
 dtype for obs_x[0] is  float32
found [1] in obs_x[1]
 key  obs_x[1] , dict_key obs_x_E0ESigma
 dtype for obs_x[1] is  float32
found [2] in obs_x[2]
 key  obs_x[2] , dict_key obs_x_E0ThetaSigma
 dtype for obs_x[2] is  float32
found [3] in obs_x[3]
 key  obs_x[3] , dict_key obs_x_DA
 dtype for obs_x[3] is  float32
found [4] in obs_x[4]
 key  obs_x[4] , dict_key obs_x_h
 dtype for obs_x[4] is  float32
found [5] in obs_x[5]
 key  obs_x[5] , dict_key o

In [17]:
branch_dict

{'tower_sum_40': dtype('float32'),
 'n_towers_40': dtype('int64'),
 'eta_pho_closest_to_ebeam': dtype('float32'),
 'e_pho_closest_to_ebeam': dtype('float64'),
 'phi_pho_closest_to_ebeam': dtype('float32'),
 'obs_x_e': dtype('float32'),
 'obs_x_E0ESigma': dtype('float32'),
 'obs_x_E0ThetaSigma': dtype('float32'),
 'obs_x_DA': dtype('float32'),
 'obs_x_h': dtype('float32'),
 'obs_x_ISigma': dtype('float32'),
 'obs_x_IDA': dtype('float32'),
 'obs_x_ThetaSigmagamma': dtype('float32'),
 'obs_x_eSigma': dtype('float32'),
 'obs_y_e': dtype('float32'),
 'obs_y_E0ESigma': dtype('float32'),
 'obs_y_E0ThetaSigma': dtype('float32'),
 'obs_y_DA': dtype('float32'),
 'obs_y_h': dtype('float32'),
 'obs_y_ISigma': dtype('float32'),
 'obs_y_IDA': dtype('float32'),
 'obs_y_ThetaSigmagamma': dtype('float32'),
 'obs_y_eSigma': dtype('float32'),
 'obs_Q2_e': dtype('float32'),
 'obs_Q2_E0ESigma': dtype('float32'),
 'obs_Q2_E0ThetaSigma': dtype('float32'),
 'obs_Q2_DA': dtype('float32'),
 'obs_Q2_h': dtype('f

In [18]:
%%time

root_file3 = uproot3.recreate( output_root_file )

root_file3['dnnout'] = uproot3.newtree( branch_dict )

root_file3['dnnout'].extend( data_dict )

CPU times: user 2.13 s, sys: 38.3 ms, total: 2.16 s
Wall time: 2.16 s
